In [1]:
# Import modules
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import Normalization

In [2]:
cosign_sim = np.load('cosine_sim_wpca.npy')
df = pd.read_csv('clean_data.csv', sep=',')

In [3]:
df = df.set_index('name')
df = df.drop(['series'], axis=1)
df = df.drop(['id'], axis=1)
df.head()

,released,rating,playtime,Action,Adventure,Arcade,Board Games,Card,Casual,Educational,...,Web,Wii,Wii U,Xbox,Xbox 360,Xbox One,Xbox Series S/X,bit.1,iOS,macOS
name,,,,,,,,,,,,,,,,,,,,,
King Arthur's Gold,0.811938,0.596708,0.002222,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Grand Theft Auto V,0.809502,0.919753,0.081111,1,1,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
The Witcher 3: Wild Hunt,0.840331,0.958848,0.051111,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
Portal 2,0.764731,0.950617,0.012222,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
Tomb Raider (2013),0.799566,0.833333,0.011111,1,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1


In [4]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
cols = list(df.columns)
#print(cols)

In [7]:
model = Sequential() 
model.add(Dense(256, activation='sigmoid', input_shape=(1992,)))
model.add(Dense(128, activation='relu')) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1992))

In [8]:
model.compile(optimizer='adam', loss='mse') 
model.fit(train_set[cols], train_set[cols], epochs=8, batch_size=32)


Epoch 1/8
246/246 [==============================] - 3s 9ms/step - loss: 0.0074
Epoch 2/8
246/246 [==============================] - 2s 9ms/step - loss: 0.0067
Epoch 3/8
246/246 [==============================] - 2s 9ms/step - loss: 0.0063
Epoch 4/8
246/246 [==============================] - 2s 6ms/step - loss: 0.0060
Epoch 5/8
246/246 [==============================] - 2s 8ms/step - loss: 0.0058
Epoch 6/8
246/246 [==============================] - 2s 9ms/step - loss: 0.0056
Epoch 7/8
246/246 [==============================] - 2s 8ms/step - loss: 0.0055
Epoch 8/8
246/246 [==============================] - 2s 8ms/step - loss: 0.0053


In [9]:
latent_factors = model.predict(df[cols])

307/307 [==============================] - 1s 2ms/step


In [33]:
# alpha is a hyperparameter that controls weight of cosign vs latent
# cosign(0) vs latent(1) weight
alpha = .4


In [34]:
latent_sim = cosine_similarity(latent_factors)
hybrid_sim = alpha * latent_sim + (1 - alpha) * cosign_sim

In [22]:
def get_recommendations(title, k):
    idx = df.index.get_loc(title)
    sim_scores = hybrid_sim[idx]
    sim_indices = sim_scores.argsort()[-k:][::-1]
    sim_titles = df.iloc[sim_indices].index
    return list(sim_titles)

In [26]:
recommendation = "Lego City Undercover"

In [35]:
get_recommendations(recommendation, 25)

['Lego City Undercover',
 'LEGO Marvel Super Heroes 2',
 'LEGO Worlds',
 'LEGO Marvel Super Heroes',
 'LEGO Jurassic World',
 'LEGO Batman 3: Beyond Gotham',
 'LEGO The Hobbit',
 'LEGO The Incredibles',
 'The LEGO Movie - Videogame',
 "LEGO Marvel's Avengers",
 'Just Die Already',
 'LEGO Batman 2 DC Super Heroes',
 'Ben 10: Power Trip',
 'Saints Row: The Third Remastered',
 'River City Girls',
 'LEGO DC Super-Villains',
 'LEGO Star Wars: The Force Awakens',
 'Overlord: Fellowship of Evil',
 'Think of the Children',
 'Darksiders Genesis',
 'Unravel Two',
 'ToeJam & Earl: Back in the Groove',
 'Tools Up!',
 'Scribblenauts Unmasked: A DC Comics Adventure',
 'Saints Row IV']

In [29]:
get_recommendations(recommendation, 25)

['Lego City Undercover',
 'LEGO Marvel Super Heroes 2',
 'LEGO Marvel Super Heroes',
 'LEGO Jurassic World',
 'LEGO The Hobbit',
 'The LEGO Movie - Videogame',
 'LEGO The Incredibles',
 'LEGO Batman 3: Beyond Gotham',
 'LEGO Worlds',
 "LEGO Marvel's Avengers",
 'LEGO Batman 2 DC Super Heroes',
 'River City Girls',
 'Just Die Already',
 'LEGO DC Super-Villains',
 'Ben 10: Power Trip',
 'Saints Row: The Third Remastered',
 'Unravel Two',
 'LEGO Star Wars: The Force Awakens',
 'SpongeBob SquarePants: Battle for Bikini Bottom — Rehydrated',
 'Guns, Gore and Cannoli 2',
 'ToeJam & Earl: Back in the Groove',
 'LEGO Star Wars - The Complete Saga',
 'Trine 4: The Nightmare Prince',
 'Think of the Children',
 'Totally Reliable Delivery Service']

In [32]:
get_recommendations(recommendation, 25)

['Lego City Undercover',
 'LEGO Marvel Super Heroes',
 'LEGO Jurassic World',
 'Nongunz: Doppelganger Edition',
 'Unravel Two',
 'SpongeBob SquarePants: Battle for Bikini Bottom — Rehydrated',
 'Trine 4: The Nightmare Prince',
 "LEGO Marvel's Avengers",
 'LEGO The Hobbit',
 'LEGO Marvel Super Heroes 2',
 'Stubbs the Zombie in Rebel Without a Pulse',
 'The LEGO Movie - Videogame',
 'Guns, Gore and Cannoli 2',
 'LEGO The Incredibles',
 'LEGO Star Wars - The Complete Saga',
 'Balan Wonderworld',
 'Strange Brigade',
 'River City Girls',
 'Super Monkey Ball: Banana Mania',
 'Speed Brawl',
 'LEGO DC Super-Villains',
 'Sniper Elite V2 Remastered',
 'Cuphead',
 'GRID Legends',
 'Totally Reliable Delivery Service']